In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/crop_train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)
train_cnt = int(train_data_cnt * 0.8)
valid_cnt = train_data_cnt - train_cnt
BATCH_SIZE = 13
CROP_LEN = 224

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def get_img(img_path, train_flag = True):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        final_img = random_crop(im_array)
    else:
        final_img = center_crop(im_array)
    
    final_img = final_img/127.5
    final_img = final_img - 1.0
    return final_img

# 读取测试数据中间 224
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/127.5
    final_img = final_img - 1.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        curr_x = [get_img(p,train_flag) for p in curr_fl]
        curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

train_gen = data_gen(train_files[:train_cnt], BATCH_SIZE, True)
valid_gen = data_gen(train_files[train_cnt:], BATCH_SIZE, False)
import math
train_step = math.ceil(train_cnt*0.5/BATCH_SIZE)
valid_step = math.ceil(valid_cnt/BATCH_SIZE)
    

{'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]}


In [2]:
# def model
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [ ]:
model_p = 'best_resnet_crop.h5'

if os.path.exists(model_p):
    model = load_model(model_p)
    #K.set_value(model.optimizer.lr, 0.0001)
    print('load',model_p)
else:
    from keras.applications.resnet50 import ResNet50
    base_model = ResNet50(input_shape=(CROP_LEN,CROP_LEN,3),include_top=False,weights=None,pooling='max')
    x = base_model.output
    x = Dense(256,activation='relu')(x)
    pred = Dense(10,activation='softmax')(x)
    model = Model(inputs=base_model.input,outputs=pred)
    print('get model')
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

load best_resnet_crop.h5


In [ ]:
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00000001)

model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=70,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )

# Epoch 00092: val_acc improved from 0.88524 to 0.89842, saving model to best_resnet_crop.h5
# 1354/1354 [==============================] - 884s 
# 653ms/step - loss: 0.1565 - acc: 0.9449 - val_loss: 0.3287 - val_acc: 0.8984
                

Epoch 1/70
1063/1354 [======================>.......] - ETA: 2:43 - loss: 0.1456 - acc: 0.9482

In [ ]:
best_model = load_model(model_p)
test_y = []
for img_p in test_files:
    tmp_x = get_test_img(img_p)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

In [ ]:
import pandas as pd
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/resnet_manip_crop.csv',index=False)

In [ ]:
# TTA
def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    im_array = im_array/127.5
    im_array = im_array - 1.0
    img_list = []
    img_list.append(center_crop(im_array))
#     for i in range(4):
#         for j in range(3):
#             a = i*72
#             b = a + 224
#             c = j*96
#             d = c + 224
#             #print(a,b,c,d)
#             img_list.append(im_array[a:b,c:d,:])
    img_list.append(im_array[0:CROP_LEN,0:CROP_LEN,:])
    img_list.append(im_array[0:CROP_LEN,512-CROP_LEN:512,:])
    img_list.append(im_array[512-CROP_LEN:512,0:CROP_LEN,:])
    img_list.append(im_array[512-CROP_LEN:512,512-CROP_LEN:512,:])
    return np.array(img_list)
tta_img = get_tta_img(test_files[0])
print(tta_img.shape)
tta_res = best_model.predict(tta_img)
print(tta_res.shape)
print(np.sum(tta_res,axis=0))

test_y = []
for img_p in test_files:
    tmp_x = get_tta_img(img_p)
    tmp_y = best_model.predict(tmp_x)
    tmp_y = np.sum(tmp_y,axis=0)
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/resnet_manip_crop_tta.csv',index=False)